In [251]:
import pandas as pd
import numpy as np

#import csv from database
movies = pd.read_csv('./database/movies_local.csv')
movies = movies.sort_values('movie')
movies = movies.set_index('movie')

#rename all columns
movies.columns = ['indication_year', 'number_indications', 'victories']

In [252]:
#convet year into numbers
movies['indication_year'] = movies['indication_year'].astype(int)
movies.head()

,indication_year,number_indications,victories
movie,,,
"$1,000 a Minute",1936,1,0
'38',1987,1,0
'Crocodile' Dundee,1987,1,0
'Round Midnight,1987,2,1
(A) Torzija [(A) Torsion],2004,1,0


# Gerando Algumas estatisticas

In [253]:
def percentage_victories(indications, victories):
    return (victories * 100) / indications
movies['percentage_victories'] = np.vectorize(percentage_victories)(movies['number_indications'], movies['victories'])

In [254]:
movies.head()

,indication_year,number_indications,victories,percentage_victories
movie,,,,
"$1,000 a Minute",1936,1,0,0.0
'38',1987,1,0,0.0
'Crocodile' Dundee,1987,1,0,0.0
'Round Midnight,1987,2,1,50.0
(A) Torzija [(A) Torsion],2004,1,0,0.0


In [352]:
bdteste = movies.iloc[-10:,:].copy()
bdteste

,indication_year,number_indications,victories,percentage_victories
movie,,,,
Zorba the Greek,1965,7,3,42.857143
Zus & Zo,2003,1,0,0.000000
images,1973,1,0,0.000000
"sex, lies, and videotape",1990,1,0,0.000000
the accountant,2002,1,1,100.000000
the end,1996,1,0,0.000000
tom thumb,1959,1,1,100.000000
À Nous la Liberté,1933,1,0,0.000000
Ådalen '31,1970,1,0,0.000000


In [353]:
import datetime
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

bkp = []

14:51:1


In [392]:
import requests
import json
api_key = 'aa043c2ae33192f63fcc150bb97816b7'


def budg_imdb_rev(mvid):
    if mvid and mvid != 0:
        url = "https://api.themoviedb.org/3/movie/"+str(mvid)+"?api_key="+api_key+"&language=en-US"

        payload = "{}"

        response = requests.request("GET", url, data=payload)
        data = json.loads(response.text)
        genres = [x['name'] for x in data['genres']]
        production_countries = [x['iso_3166_1'] for x in data['production_countries']]
        return  np.array([data['imdb_id'], data['budget'], data['revenue'], genres, production_countries], dtype=object)
    
    return  np.array([False, False, False, False, False], dtype=object)

def find_movie(mv, yr):
    
    now = datetime.datetime.now()
    url = "https://api.themoviedb.org/3/search/movie?api_key="+api_key+"&language=en-US&query="+mv
    url= url+"&primary_release_year="+str(yr-1)
    payload = "{}"

    response = requests.request("GET", url, data=payload)
    data = json.loads(response.text)
    
    print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))
    print(mv + ' | ' +str(data['total_results']))
    
    if len(data['results']) == 1:
        return  data['results'][0]['id']
    
    elif data['results'] and data['total_results'] > 1:
        lmmv = []
        
        for mvs in data['results']:
            if mvs['title'] == mv:
                lmmv.append(mvs)
                
        if len(lmmv) == 1:
            return  lmmv[0]['id']
        elif len(lmmv) > 1:
            return  False
        
        return  False
   
    return  False

In [420]:
#divide movies database for analisis

moviesdv=[]
for x in range(1,159):
    moviesdv.append(movies.iloc[int(len(movies.index.values)/159)*(x-1):int(len(movies.index.values)/159)*x,:].copy())

len(moviesdv[0])

30

In [421]:
for idx, df in enumerate(moviesdv):
    print(str(idx)+' | ===================================================================================')
    df['tmdb_id'] =  np.vectorize(find_movie)(df.index.values, df['indication_year'])

0 | ===================================================================================
15:58:3
$1,000 a Minute | 1
15:58:8
$1,000 a Minute | 1
15:58:10
'38' | 1
15:58:10
'Crocodile' Dundee | 1
15:58:11
'Round Midnight | 1
15:58:13
(A) Torzija [(A) Torsion] | 0
15:58:17
...And Justice for All | 1
15:58:19
10 | 10
15:58:23
102 Dalmatians | 1
15:58:24
12 | 11
15:58:26
12 Angry Men | 1
15:58:28
12 Monkeys | 1
15:58:29
12 Years a Slave | 1
15:58:33
127 Hours | 1
15:58:35
12:01 PM | 1
15:58:36
13 Hours: The Secret Soldiers of Benghazi | 1
15:58:37
13th | 2
15:58:37
140 Days under the World | 1
15:58:39
1776 | 1
15:58:44
1848 | 0
15:58:45
1941 | 1
15:58:47
20 Feet from Stardom | 1
15:58:49
20,000 Leagues under the Sea | 1
15:58:49
2001: A Space Odyssey | 1
15:58:50
2010 | 1
15:58:54
20th Century Women | 1
15:58:55
21 Grams | 1
15:58:58
24-Hour Alert | 1
15:58:59
3 Misses | 0
15:59:1
3:10 to Yuma | 1
15:59:1
3rd Ave. El | 1
1 | =================================================================

16:3:28
Almost Famous | 1
16:3:29
Almost Famous | 1
16:3:30
Aloma of the South Seas | 1
16:3:30
Alone Yet Not Alone | 1
16:3:31
Along the Rainbow Trail | 0
16:3:32
Alsino and the Condor | 0
16:3:33
Altered States | 1
16:3:33
Alvorada (Brazil's Changing Face) | 1
16:3:34
Always a New Beginning | 0
16:3:35
Always in My Heart | 1
16:3:35
Ama Girls | 1
16:3:36
Amadeus | 1
16:3:37
Amarcord | 0
16:3:37
Amarcord | 0
16:3:38
Amazon | 3
16:3:39
Amazon Diary | 1
16:3:39
America America | 4
16:3:40
American Beauty | 1
16:3:41
American Dream | 2
16:3:41
American Gangster | 1
16:3:42
American Graffiti | 1
16:3:42
American History X | 1
16:3:43
American Hustle | 1
16:3:44
American Shoeshine | 1
16:3:44
American Sniper | 1
16:3:45
American Splendor | 1
16:3:46
Americas in Transition | 0
16:3:46
Amistad | 1
16:3:47
Amores Perros | 1
16:3:48
Amour | 24
16:3:48
Amphibious Fighters | 1
9 | ===================================================================================
16:3:49
Amy | 5
16:3:50
Amy | 5


16:6:46
Birdman or (The Unexpected Virtue of Ignorance) | 1
16:6:46
Birdnesters of Thailand (aka 'Shadow Hunters') | 0
16:6:47
Birds Anonymous | 1
16:6:48
Birds Do It, Bees Do It | 0
16:6:48
Birth of the Blues | 1
16:6:49
Birthday Boy | 1
16:6:50
Bismarck Convoy Smashed | 0
16:6:50
Bite the Bullet | 1
16:6:51
Bitter Rice | 0
16:6:52
Bitter Sweet | 1
16:6:52
Biutiful | 1
16:6:53
Black Fox | 1
16:6:54
Black Fury | 1
16:6:54
Black Hawk Down | 1
16:6:55
Black Legion | 2
17 | ===================================================================================
16:6:56
Black Narcissus | 1
16:6:56
Black Narcissus | 1
16:6:57
Black Orpheus | 1
16:6:58
Black Rain | 3
16:6:58
Black Rider (Schwarzfahrer) | 1
16:6:59
Black Swan | 1
16:7:0
Black and White in Color | 1
16:7:0
Blackboard Jungle | 1
16:7:1
Blackfly | 1
16:7:2
Blackwood | 1
16:7:2
Blade Runner | 1
16:7:3
Blake | 2
16:7:4
Blaze | 1
16:7:4
Blaze Busters | 1
16:7:5
Blazing Saddles | 1
16:7:6
Bless the Beasts & Children | 1
16:7:6
Blind Vays

16:9:49
Children of Darkness | 1
16:9:50
Children of Fate | 1
16:9:51
Children of Heaven | 0
16:9:51
Children of Mars | 1
16:9:52
Children of Men | 1
16:9:53
Children of Nature | 1
16:9:53
Children of Paradise | 0
16:9:54
Children of a Lesser God | 1
16:9:55
Chile: Hasta Cuando? | 1
16:9:55
Chimps: So Like Us | 1
16:9:56
China's Unnatural Disaster: The Tears of Sichuan Province | 1
16:9:56
Chinatown | 2
16:9:57
Chip An' Dale | 1
16:9:58
Chitty Chitty Bang Bang | 1
16:9:58
Chocolat | 2
16:9:59
Chopin's Musical Moments | 0
16:10:0
Christ among the Primitives | 1
16:10:0
Christmas Cracker | 0
16:10:1
Christmas Holiday | 1
16:10:2
Christmas under Fire | 1
16:10:2
Christo's Valley Curtain | 0
16:10:3
Christopher Crumpet | 1
16:10:4
Churchill's Island | 1
16:10:4
Cimarron | 1
16:10:5
Cimarron | 1
16:10:6
Cinderella | 2
16:10:6
Cinderella | 1
16:10:7
Cinderella Horse | 0
26 | ===================================================================================
16:10:7
Cinderella Liberty | 1
16:

16:12:36
Dodes'ka-Den | 0
16:12:37
Dodsworth | 1
16:12:37
Dog Day Afternoon | 1
16:12:38
Dog, Cat and Canary | 0
16:12:39
Dogtooth | 0
16:12:39
Doing Time: Life inside the Big House | 1
16:12:40
Dolphins | 1
16:12:40
Don Juan DeMarco | 0
16:12:41
Don't | 7
16:12:42
Don't Mess with Bill | 1
16:12:42
Don't Talk | 1
16:12:43
Don't Tell | 1
16:12:44
Donald in Mathmagic Land | 1
16:12:44
Donald's Crime | 1
16:12:45
Donnie Brasco | 1
16:12:45
Double Feature | 1
16:12:46
Double Indemnity | 1
16:12:47
Double or Nothing | 1
16:12:47
Doubletalk | 1
34 | ===================================================================================
16:12:48
Doubt | 3
16:12:49
Doubt | 3
16:12:49
Down Argentine Way | 1
16:12:50
Down and Out in America | 1
16:12:51
Down on the Farm | 1
16:12:51
Down on the Waterfront | 1
16:12:52
Downfall | 2
16:12:53
Dr. Cyclops | 1
16:12:53
Dr. Ehrlich's Magic Bullet | 1
16:12:54
Dr. Jekyll and Mr. Hyde | 1
16:12:54
Dr. Jekyll and Mr. Hyde | 0
16:12:55
Dr. Jekyll and Mr. Mous

...

16:52:42
The Hobbit: The Battle of the Five Armies | 1
16:52:43
The Hobbit: The Desolation of Smaug | 1
16:52:43
The Hole | 2
16:52:44
The Horse Whisperer | 1
16:52:44
The Horse with the Flying Tail | 1
16:52:45
The Horse's Mouth | 1
16:52:46
The Hospital | 1
16:52:46
The Hot Rock | 1
16:52:47
The Hours | 3
16:52:48
The House That Ananda Built | 1
16:52:48
The House That Jack Built | 1
16:52:49
The House of Rothschild | 1
16:52:49
The House of the Seven Gables | 1
16:52:50
The House on 92nd Street | 1
16:52:51
The House on Chelouche Street | 1
16:52:51
The House without a Name | 1
16:52:52
The Howards of Virginia | 1
16:52:53
The Human Comedy | 1
16:52:53
The Human Dutch | 0
16:52:54
The Hunchback of Notre Dame | 5
16:52:55
The Hunchback of Notre Dame | 1
16:52:55
The Hunt | 16
16:52:56
The Hunt for Red October | 1
16:52:57
The Hunting Ground | 1
122 | ===================================================================================
16:52:57
The Hurricane | 1
16:52:58
The Hurricane |

16:55:11
The Moon Is Blue | 1
16:55:12
The Moon and Sixpence | 0
16:55:13
The Moon and the Son: An Imagined Conversation | 1
16:55:16
The More the Merrier | 1
16:55:16
The Morning After | 1
16:55:18
The Morning Spider | 1
16:55:19
The Most Dangerous Man in America: Daniel Ellsberg and the Pentagon Papers | 1
16:55:22
The Motorcycle Diaries | 1
16:55:23
The Mudlark | 0
16:55:24
The Mummy | 5
16:55:24
The Muppet Movie | 1
16:55:25
The Muppets | 1
16:55:26
The Muppets Take Manhattan | 1
129 | ===================================================================================
16:55:26
The Mushroom Club | 1
16:55:27
The Mushroom Club | 1
16:55:27
The Music Box | 1
16:55:29
The Music Teacher | 1
16:55:29
The Mysterious Geographic Explorations of Jasper Morello | 1
16:55:30
The Naked City | 1
16:55:31
The Naked Eye | 1
16:55:31
The Naked Prey | 0
16:55:32
The Naked Spur | 1
16:55:32
The Narrow Margin | 1
16:55:33
The Nasty Girl | 1
16:55:34
The Natural | 1
16:55:34
The Navy Comes Through | 1


16:57:49
The Shop on Main Street | 0
16:57:49
The Shop on Main Street | 0
16:57:50
The Shop on Main Street | 1
16:57:50
The Shore | 2
16:57:51
The Sign of the Cross | 0
16:57:52
The Silence | 1
16:57:52
The Silence of the Lambs | 1
16:57:53
The Silent Revolution | 1
16:57:54
The Silent World | 1
16:57:54
The Silver Chalice | 1
16:57:55
The Sin of Madelon Claudet | 0
16:57:56
The Singing Nun | 1
16:57:56
The Sixth Sense | 1
16:57:57
The Sky's the Limit | 1
16:57:57
The Slender Thread | 1
16:57:58
The Slipper and the Rose--The Story of Cinderella | 0
16:57:59
The Smiling Lieutenant | 0
16:57:59
The Snake Pit | 1
16:58:0
The Sniper | 1
16:58:1
The Snowman | 1
16:58:1
The Snows of Kilimanjaro | 1
16:58:2
The Social Network | 1
16:58:3
The Solar Film | 0
16:58:3
The Solid Gold Cadillac | 1
16:58:4
The Son of Monte Cristo | 0
16:58:4
The Song of Bernadette | 1
16:58:5
The Sorrow and the Pity | 0
16:58:6
The Sound of Music | 1
16:58:6
The Southerner | 1
16:58:7
The Spanish Main | 1
16:58:8
Th

17:0:19
They Were Expendable | 1
17:0:19
They're Always Caught | 1
17:0:20
Thin Ice | 1
17:0:21
Thirteen | 2
17:0:21
Thirty Million Letters | 1
17:0:22
Thirty Seconds over Tokyo | 1
17:0:23
This Charming Man (Der Er En Yndig Mand) | 0
17:0:23
This Is Cinerama | 0
17:0:24
This Is the Army | 1
17:0:25
This Land Is Mine | 1
17:0:25
This Love of Ours | 1
17:0:26
This Mechanical Age | 1
17:0:26
This Sporting Life | 1
17:0:27
This Tiny World | 1
17:0:28
This Way Up | 1
17:0:28
This Woman Is Mine | 1
17:0:29
This above All | 1
17:0:30
Thoroughly Modern Millie | 1
17:0:30
Those Magnificent Men in Their Flying Machines | 1
144 | ===================================================================================
17:0:31
Thoth | 0
17:0:32
Thoth | 0
17:0:33
Thousands Cheer | 1
17:0:33
Three | 7
17:0:34
Three Brothers | 1
17:0:35
Three Coins in the Fountain | 1
17:0:35
Three Comrades | 1
17:0:36
Three Days of the Condor | 1
17:0:37
Three Is a Family | 0
17:0:38
Three Kisses | 1
17:0:38
Three Little

17:3:20
Walter Wanger's Vogues of 1938 | 0
17:3:20
Waltz with Bashir | 1
17:3:21
Waltzing Regitze | 1
17:3:21
Wanted | 6
17:3:22
Wanted, a Master | 1
17:3:24
War Clouds in the Pacific | 0
17:3:25
War Department Report | 1
17:3:25
War Horse | 1
17:3:27
War Photographer | 1
17:3:28
War Witch | 2
17:3:28
War and Peace | 0
17:3:29
War and Peace | 1
17:3:30
War of the Worlds | 4
17:3:31
War/Dance | 1
17:3:31
WarGames | 1
17:3:32
Warrior | 18
17:3:33
Wartime Romance | 0
17:3:33
Wasp | 0
17:3:34
Waste Land | 1
17:3:35
Wat's Pig | 1
17:3:36
Watani: My Homeland | 1
17:3:36
Watch on the Rhine | 1
17:3:37
Water | 24
17:3:38
Water Birds | 1
17:3:38
Water Trix | 1
17:3:39
Water--Friend or Enemy | 1
17:3:40
Waterloo Bridge | 1
17:3:40
Waterworld | 1
17:3:41
Way Down South | 1
17:3:41
Way Out West | 1
153 | ===================================================================================
17:3:42
We Can't Live without Cosmos | 0
17:3:43
We Can't Live without Cosmos | 0
17:3:43
We Refuse to Die | 1
1

In [422]:
moviesdv[0]

,indication_year,number_indications,victories,percentage_victories,tmdb_id
movie,,,,,
"$1,000 a Minute",1936,1,0,0.000000,218388
'38',1987,1,0,0.000000,42016
'Crocodile' Dundee,1987,1,0,0.000000,9671
'Round Midnight,1987,2,1,50.000000,14670
(A) Torzija [(A) Torsion],2004,1,0,0.000000,0
...And Justice for All,1980,2,0,0.000000,17443
10,1980,2,0,0.000000,9051
102 Dalmatians,2001,1,0,0.000000,10481
12,2008,1,0,0.000000,20714


In [424]:
moviesdvs = pd.concat(moviesdv)

In [425]:
moviesdvs.to_csv('./database/movies_TMDB_PI1.csv')

In [426]:
moviesdvs_not_tmdb= moviesdvs.loc[moviesdvs['tmdb_id'] == 0]

len(moviesdvs_not_tmdb)

714

In [ ]:
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

In [ ]:
for idx, df in enumerate(moviesdv):
    print(str(idx)+' | ===================================================================================')
    #df['tmdb_id'] =  np.vectorize(find_movie)(df.index.values, df['indication_year'])
    df[['imdb_id','budget','revenue', 'genres', 'production_countries']] = pd.DataFrame(list(movies['tmdb_id'].apply(budg_imdb_rev)), index=movies.index)


In [ ]:

#movies[['imdb_id','budget','revenue', 'genres', 'production_countries']] = pd.DataFrame(list(movies['tmdb_id'].apply(budg_imdb_rev)), index=movies.index)


In [ ]:
#movies.to_csv('./database/movies_TMDB_PI2.csv')

In [ ]:
now = datetime.datetime.now()

print(str(now.hour)+':'+str(now.minute)+':'+str(now.second))

In [ ]:
#https://developers.themoviedb.org/3/movies/get-movie-details
#https://github.com/richardasaurus/imdb-pie#get_title_credits
#https://market.mashape.com/marcalencc/metacritic#
movies.head()